# Derivative_Project01: Setting up Derivative_project - Data prep

In [1]:
project = 'derivative_project01'
import antropy as ant
from antropy import lziv_complexity
import matplotlib
#matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, show
import pathlib
import mne
import os
import os.path as op
from mne import viz
from mne import io
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)
import zipfile
import sys
import pandas as pd
import numpy as np
from IPython.utils import io
import neurokit2 as nk
import scipy
from scipy import stats
import scipy.stats
import seaborn as sns
import pickle
import itertools
from whatever import *

# Participants

In [2]:
len(sorted(os.listdir('./sourcedata')))

71

### Creating new data location

In [3]:
if not os.path.exists('./derivative_project05/data'):
    os.mkdir('./derivative_project05/data')
else:
    print ('data folder already exists')

data folder already exists


### Choosing subjects and files 

In [112]:
# Final Subject List
sub_final = []
for i in sorted(os.listdir('./sourcedata')): 
    if 'CCTC06NETICU' not in i:
        sub_final.append(i) 
        
# Subfolders/Tasks
fol_names = []
removed = []
for i in sub_final:
    print(f'{i}:')
    subfolders = os.listdir(f'./sourcedata/{i}/eeg')
    for s in subfolders:
        if '(failed)' not in s and 'tennis' not in s and 'ERP' not in s and 'sub-' in s:
            print(f'    -{s}')
            fol_names.append(s) 
        else:
            removed.append(s) 
            print ('-----------ERROR---------------------------------------', s)


sub-CCTC03:
    -sub-CCTC03_task-taken_eeg
    -sub-CCTC03_task-rest02_eeg
    -sub-CCTC03_task-taken02_eeg
    -sub-CCTC03_task-rest_eeg
sub-CCTC04:
    -sub-CCTC04_task-restredo_eeg
    -sub-CCTC04_task-taken_eeg
    -sub-CCTC04_task-rest_eeg
sub-CCTC05:
    -sub-CCTC05_task-RS_ST_Taken_eeg
-----------ERROR--------------------------------------- sub-CCTC05_task-ERP_eeg
sub-CCTC07:
    -sub-CCTC07_task-rest_eeg
    -sub-CCTC07_task-taken_eeg
-----------ERROR--------------------------------------- sub-CCTC07_task-ERP_eeg
sub-CCTC07NETICU:
    -sub-CCTC07NETICU_task-sedon1rest_eeg
    -sub-CCTC07NETICU_task-sedoffrest_eeg
    -sub-CCTC07NETICU_task-sedofftaken_eeg
    -sub-CCTC07NETICU_task-sedon1taken_eeg
sub-HC01:
    -sub-HC01_task-ST_eeg
    -sub-HC01_task-IT_eeg
sub-HC02:
    -sub-HC02_task-Taken_eeg
sub-HC03:
    -sub-HC03_task-Taken_eeg
sub-HC04:
    -sub-HC04_task-Taken_eeg
sub-HC05:
    -sub-HC05_task-Taken_eeg
sub-HC06:
    -sub-HC06_task-Taken_eeg
sub-HC07:
    -sub-HC07_task

In [118]:
#removing the 'sub-' in sub_final = p_list 
p_list = []
for i in sub_final:
    p_list.append(i[4:])
    
len(p_list)

13

In [119]:
p_list, fol_names

(['CCTC07',
  'CCTC07NETICU',
  'MSICU32',
  'MSICU38',
  'MSICU42',
  'MSICU43',
  'MSICU47',
  'NETICU31',
  'NETICU33',
  'NETICU38',
  'NETICU40',
  'NETICU41',
  'NETICU46'],
 ['sub-CCTC07_task-rest_eeg',
  'sub-CCTC07_task-taken_eeg',
  'sub-CCTC07NETICU_task-sedon1rest_eeg',
  'sub-CCTC07NETICU_task-sedoffrest_eeg',
  'sub-CCTC07NETICU_task-sedofftaken_eeg',
  'sub-CCTC07NETICU_task-sedon1taken_eeg',
  'sub-MSICU32_task-rest_eeg',
  'sub-MSICU32_task-taken_eeg',
  'sub-MSICU38_task-taken_eeg',
  'sub-MSICU38_task-rest_eeg',
  'sub-MSICU42_task-rest_eeg',
  'sub-MSICU42_task-taken_eeg',
  'sub-MSICU43_task-rest_eeg',
  'sub-MSICU43_task-taken_eeg',
  'sub-MSICU47_task-rest_eeg',
  'sub-MSICU47_task-taken_eeg',
  'sub-NETICU31_task-sedofftaken_eeg',
  'sub-NETICU31_task-sedon2rest_eeg',
  'sub-NETICU31_task-sedon1taken_eeg',
  'sub-NETICU31_task-sedon1rest_eeg',
  'sub-NETICU33_task-sedoffrest_eeg',
  'sub-NETICU33_task-sedon1taken_eeg',
  'sub-NETICU33_task-sedon1rest_eeg',
  'su

## Sending files to new location


In [122]:
### example with one participant file ###
p_list = ['MSICU32']

In [123]:
 ###################################
# Sending the files to new location (data) #
 ###################################

for ID in p_list:
    print (f'\n________________________________________________________________________________________________\n____________________________________________{ID}______________________________________________\n')
#------------------------------------------------------------------------------------------------------------------#    
    
    # Iterate over list of conditions for each participant
    tlist = vars()[ID+'_tlist']
    for c in tlist:
        print ('\n___________',c,'___________')
        #print ('(1) Searching for working directorys.')
        source_path = f'./sourcedata/sub-{ID}/eeg/sub-{ID}_task-{c}_eeg' 
        fname  = (os.listdir(source_path)[0])[:-4] #using string sliciung OR use replace Ex: fname=file.replace(".zip", "")
        #create new directory
        new_path = f'./{project}/data/sub-{ID}/sub-{ID}_task-{c}_eeg'
        if not os.path.exists(new_path):
            os.makedirs(new_path)
            #print('    - No directory found. \n    - New directory made.')
        else:
            print('    - This directory already exists. \n    - Continue with existing file.') 
        #unzip source files to the participants new directory
        #print('(2) Unzipping files into prepoc directory.')         
        if not os.path.exists(f'{new_path}/{fname}'):
            with zipfile.ZipFile(f'{source_path}/{fname}.zip', 'r') as zip_ref:
                zip_ref.extractall(new_path)
                #print (f'    - sub-{ID}_task-{c} sourcefile was unzipped to the participants working directory.')
        else:
            print(f'    - Cannot unzip files because {fname} already exists.\n    - Continue with existing file.')
        # Rename to .mff         
        if not os.path.exists(f'./{new_path}/{fname}.mff'):
            os.rename(f'{new_path}/{fname}', f'{new_path}/{fname}.mff')
            #print(f'    - sub-{ID}_task-{c}_eeg/{fname}.mff was renamed with .mff suffix')
        else:
            print(f'    - Cant rename file to .mff because {fname}.mff file already exisits. \n    - Continuing with existing file')
        
#---------------------------------------------------------------------------------------------------#        
        # Loading metadata
        #print(f'(3) Downsampling and loading data for {ID}_{c}')
        data_path = f'{new_path}/{fname}.mff'
        with io.capture_output() as captured:
            raw = mne.io.read_raw_egi(data_path)#, preload=True)
        # Downsampling to 250Hz
        data = raw.copy()
        raw_sfreq = raw.info['sfreq']
        data_sfreq = data.info['sfreq']
        #print(f'    - The raw sampling frequincy is {raw_sfreq}Hz.')
        if data_sfreq > 250:
            with io.capture_output() as captured:
                data.resample(250)
            print(f'    > Data was downsampled to 250Hz from {raw_sfreq}')
        if data_sfreq < 250: 
            print(f'    > Error: Data sampling frequency is {raw_sfreq}Hz.')
            answer = input('      - Do you want to continue(yes/no)?:')
            if answer.lower().startswith("y"):
                print("      - ok, carry on then")
            elif answer.lower().startswith("n"):
                print("      - ok, sayonnara")
                sys.exit()
        # loading in data      
        #print('    - Loading data into memory...')
        with io.capture_output() as captured: 
            data.load_data() 
        # Timeing
        meas_data = data.info['meas_date']
        print('(1) Data timing...', 
              f'\n    - {fname} ----> {meas_data}',
              f'\n    - {data}',
              f'\n    - time(s): {(data.last_samp+1)/250}',
              f'\n    - first_samp-last_samp: {data.first_samp} - {data.last_samp}')
        seconds = (data.last_samp+1)/250
        seconds = seconds % (24 * 3600)
        hour = seconds // 3600
        seconds %= 3600
        minutes = seconds // 60
        seconds %= 60
        print("    - Duration = %dhr : %02dmin : %02dsec" % (hour, minutes, seconds))
    ############################################################################################
    #EVENTS
    ############################################################################################
        print('(2) Trigger Channels')
        triggers = []
        for i in data.ch_names:
            if not 'E' in i:
                if not 'sync' in i:
                    if not 'STI' in i:
                        if not 'V' in i:
                            print ("    - Trigger ch.",i)
                            triggers.append(i)
        #
        with io.capture_output() as captured:
            events = mne.find_events(data, stim_channel='STI 014', initial_event=True)
        for i in range(events.shape[0]):
            vars()['event'+str(i+1)] = events[i,0]
            print (f' - {events[i,0]} / 250 = {(events[i,0])/250}')



________________________________________________________________________________________________
____________________________________________MSICU32______________________________________________


___________ rest ___________
    - This directory already exists. 
    - Continue with existing file.
    - Cant rename file to .mff because Session 20240220 2041.mff file already exisits. 
    - Continuing with existing file
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240220 2041 ----> 2024-02-21 01:44:07.407000+00:00 
    - <RawMff | signal1.bin, 131 x 90668 (362.7 s), ~90.8 MB, data loaded> 
    - time(s): 362.672 
    - first_samp-last_samp: 0 - 90667
    - Duration = 0hr : 06min : 02sec
(2) Trigger Channels

___________ taken ___________
(1) Data timing... 
    - Session 20240220 2022 ----> 2024-02-21 01:30:46.400000+00:00 
    - <RawMff | signal1.bin, 135 x 160839 (643.4 s), ~165.8 MB, data loaded> 
    - time(s): 643.356 
    - first_samp-last_s

#### Previous Output with other particpants.


________________________________________________________________________________________________
____________________________________________CCTC03______________________________________________


___________ taken ___________
(1) Data timing... 
    - Session 20010114 0336 ----> 2001-01-14 08:44:31.092000+00:00 
    - <RawMff | signal1.bin, 133 x 171170 (684.7 s), ~173.9 MB, data loaded> 
    - time(s): 684.68 
    - first_samp-last_samp: 0 - 171169
    - Duration = 0hr : 11min : 24sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 964 / 250 = 3.856
 - 89705 / 250 = 358.82

___________ rest02 ___________
(1) Data timing... 
    - Session 20010118 0013 ----> 2001-01-18 05:23:08.496000+00:00 
    - <RawMff | signal1.bin, 132 x 97510 (390.0 s), ~98.4 MB, data loaded> 
    - time(s): 390.04 
    - first_samp-last_samp: 0 - 97509
    - Duration = 0hr : 06min : 30sec
(2) Trigger Channels
    - Trigger ch. 12  
 - 1499 / 250 = 5.996

___________ taken02 ___________
(1) Data timing... 
    - Session 20010118 0046 ----> 2001-01-18 05:48:01.580000+00:00 
    - <RawMff | signal1.bin, 133 x 170857 (683.4 s), ~173.5 MB, data loaded> 
    - time(s): 683.428 
    - first_samp-last_samp: 0 - 170856
    - Duration = 0hr : 11min : 23sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 949 / 250 = 3.796
 - 89670 / 250 = 358.68

___________ rest ___________
(1) Data timing... 
    - Session 20010114 0307 ----> 2001-01-14 08:20:31.100000+00:00 
    - <RawMff | signal1.bin, 131 x 99797 (399.2 s), ~99.9 MB, data loaded> 
    - time(s): 399.188 
    - first_samp-last_samp: 0 - 99796
    - Duration = 0hr : 06min : 39sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________CCTC04______________________________________________


___________ restredo ___________
(1) Data timing... 
    - Session 20010219 0100 ----> 2001-02-19 06:04:58.056000+00:00 
    - <RawMff | signal1.bin, 132 x 97572 (390.3 s), ~98.4 MB, data loaded> 
    - time(s): 390.288 
    - first_samp-last_samp: 0 - 97571
    - Duration = 0hr : 06min : 30sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1006 / 250 = 4.024

___________ taken ___________
(1) Data timing... 
    - Session 20010219 0042 ----> 2001-02-19 05:44:29.564000+00:00 
    - <RawMff | signal1.bin, 133 x 168720 (674.9 s), ~171.4 MB, data loaded> 
    - time(s): 674.88 
    - first_samp-last_samp: 0 - 168719
    - Duration = 0hr : 11min : 14sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 965 / 250 = 3.86
 - 86863 / 250 = 347.452

___________ rest ___________
(1) Data timing... 
    - Session 20010219 0028 ----> 2001-02-19 05:32:18.128000+00:00 
    - <RawMff | signal1.bin, 132 x 97659 (390.6 s), ~98.5 MB, data loaded> 
    - time(s): 390.636 
    - first_samp-last_samp: 0 - 97658
    - Duration = 0hr : 06min : 30sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1518 / 250 = 6.072

________________________________________________________________________________________________
____________________________________________CCTC05______________________________________________


___________ RS_ST_Taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230516 1650 ----> 2023-05-16 21:59:33.873000+00:00 
    - <RawMff | signal1.bin, 137 x 244545 (978.2 s), ~255.8 MB, data loaded> 
    - time(s): 978.18 
    - first_samp-last_samp: 0 - 244544
    - Duration = 0hr : 16min : 18sec
(2) Trigger Channels
    - Trigger ch. 59  
    - Trigger ch. 69  
    - Trigger ch. 62  
    - Trigger ch. 72  
    - Trigger ch. 60  
    - Trigger ch. 70  
 - 1609 / 250 = 6.436
 - 84128 / 250 = 336.512
 - 86666 / 250 = 346.664
 - 162330 / 250 = 649.32
 - 164875 / 250 = 659.5
 - 242923 / 250 = 971.692

________________________________________________________________________________________________
____________________________________________HC01______________________________________________


___________ ST ___________
(1) Data timing... 
    - Session 20161123 1429 ----> 2016-11-23 19:49:42.096000+00:00 
    - <RawMff | signal1.bin, 133 x 228821 (915.3 s), ~232.4 MB, data loaded> 
    - time(s): 915.284 
    - first_samp-last_samp: 0 - 228820
    - Duration = 0hr : 15min : 15sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 12  
 - 3036 / 250 = 12.144
 - 128092 / 250 = 512.368

___________ IT ___________
(1) Data timing... 
    - Session 20160308 1514 ----> 2016-03-08 20:42:37.112000+00:00 
    - <RawMff | signal1.bin, 133 x 357782 (1431.1 s), ~363.2 MB, data loaded> 
    - time(s): 1431.128 
    - first_samp-last_samp: 0 - 357781
    - Duration = 0hr : 23min : 51sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2432 / 250 = 9.728
 - 272682 / 250 = 1090.728

________________________________________________________________________________________________
____________________________________________HC02______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160520 1242 ----> 2016-05-20 17:44:59.212000+00:00 
    - <RawMff | signal1.bin, 133 x 262309 (1049.2 s), ~266.3 MB, data loaded> 
    - time(s): 1049.236 
    - first_samp-last_samp: 0 - 262308
    - Duration = 0hr : 17min : 29sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2310 / 250 = 9.24
 - 160262 / 250 = 641.048

________________________________________________________________________________________________
____________________________________________HC03______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160524 0942 ----> 2016-05-24 15:14:19.604000+00:00 
    - <RawMff | signal1.bin, 134 x 404568 (1618.3 s), ~413.8 MB, data loaded> 
    - time(s): 1618.272 
    - first_samp-last_samp: 0 - 404567
    - Duration = 0hr : 26min : 58sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 11  
    - Trigger ch. 12  
 - 2831 / 250 = 11.324
 - 178964 / 250 = 715.856
 - 309102 / 250 = 1236.408

________________________________________________________________________________________________
____________________________________________HC04______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160527 1145 ----> 2016-05-27 16:49:10.816000+00:00 
    - <RawMff | signal1.bin, 134 x 299920 (1199.7 s), ~306.8 MB, data loaded> 
    - time(s): 1199.68 
    - first_samp-last_samp: 0 - 299919
    - Duration = 0hr : 19min : 59sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2349 / 250 = 9.396
 - 118235 / 250 = 472.94
 - 211461 / 250 = 845.844

________________________________________________________________________________________________
____________________________________________HC05______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160530 1318 ----> 2016-05-30 18:36:37.464000+00:00 
    - <RawMff | signal1.bin, 134 x 456930 (1827.7 s), ~467.3 MB, data loaded> 
    - time(s): 1827.72 
    - first_samp-last_samp: 0 - 456929
    - Duration = 0hr : 30min : 27sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2584 / 250 = 10.336
 - 130584 / 250 = 522.336
 - 249684 / 250 = 998.736
 - 357717 / 250 = 1430.868

________________________________________________________________________________________________
____________________________________________HC06______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160615 1350 ----> 2016-06-15 19:04:22.716000+00:00 
    - <RawMff | signal1.bin, 134 x 331370 (1325.5 s), ~339.0 MB, data loaded> 
    - time(s): 1325.48 
    - first_samp-last_samp: 0 - 331369
    - Duration = 0hr : 22min : 05sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2621 / 250 = 10.484
 - 126705 / 250 = 506.82
 - 235987 / 250 = 943.948

________________________________________________________________________________________________
____________________________________________HC07______________________________________________


___________ Taken ___________

(1) Data timing... 
    - Session 20160617 1342 ----> 2016-06-17 19:04:57.788000+00:00 
    - <RawMff | signal1.bin, 134 x 310421 (1241.7 s), ~317.5 MB, data loaded> 
    - time(s): 1241.684 
    - first_samp-last_samp: 0 - 310420
    - Duration = 0hr : 20min : 41sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2665 / 250 = 10.66
 - 111296 / 250 = 445.184
 - 214258 / 250 = 857.032

________________________________________________________________________________________________
____________________________________________HC08______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160617 1618 ----> 2016-06-17 21:43:54.144000+00:00 
    - <RawMff | signal1.bin, 134 x 309482 (1237.9 s), ~316.6 MB, data loaded> 
    - time(s): 1237.928 
    - first_samp-last_samp: 0 - 309481
    - Duration = 0hr : 20min : 37sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2033 / 250 = 8.132
 - 110966 / 250 = 443.864
 - 215739 / 250 = 862.956

________________________________________________________________________________________________
____________________________________________HC09______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160621 1415 ----> 2016-06-21 19:33:12.100000+00:00 
    - <RawMff | signal1.bin, 134 x 304871 (1219.5 s), ~311.9 MB, data loaded> 
    - time(s): 1219.484 
    - first_samp-last_samp: 0 - 304870
    - Duration = 0hr : 20min : 19sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2640 / 250 = 10.56
 - 107138 / 250 = 428.552
 - 211844 / 250 = 847.376

________________________________________________________________________________________________
____________________________________________HC10______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160622 1550 ----> 2016-06-22 21:04:32.204000+00:00 
    - <RawMff | signal1.bin, 134 x 312207 (1248.8 s), ~319.4 MB, data loaded> 
    - time(s): 1248.828 
    - first_samp-last_samp: 0 - 312206
    - Duration = 0hr : 20min : 48sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2646 / 250 = 10.584
 - 118606 / 250 = 474.424
 - 219704 / 250 = 878.816

________________________________________________________________________________________________
____________________________________________HC11______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160624 1158 ----> 2016-06-24 17:18:48.496000+00:00 
    - <RawMff | signal1.bin, 134 x 306232 (1224.9 s), ~313.3 MB, data loaded> 
    - time(s): 1224.928 
    - first_samp-last_samp: 0 - 306231
    - Duration = 0hr : 20min : 24sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2694 / 250 = 10.776
 - 108052 / 250 = 432.208
 - 213503 / 250 = 854.012

________________________________________________________________________________________________
____________________________________________HC12______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160624 1319 ----> 2016-06-24 18:34:27.356000+00:00 
    - <RawMff | signal1.bin, 134 x 315995 (1264.0 s), ~323.2 MB, data loaded> 
    - time(s): 1263.98 
    - first_samp-last_samp: 0 - 315994
    - Duration = 0hr : 21min : 03sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2039 / 250 = 8.156
 - 115728 / 250 = 462.912
 - 222752 / 250 = 891.008

________________________________________________________________________________________________
____________________________________________HC13______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160705 1347 ----> 2016-07-05 19:02:45.400000+00:00 
    - <RawMff | signal1.bin, 134 x 310319 (1241.3 s), ~317.4 MB, data loaded> 
    - time(s): 1241.276 
    - first_samp-last_samp: 0 - 310318
    - Duration = 0hr : 20min : 41sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2702 / 250 = 10.808
 - 106453 / 250 = 425.812
 - 217384 / 250 = 869.536

________________________________________________________________________________________________
____________________________________________HC14______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20160706 1321 ----> 2016-07-06 18:32:17.620000+00:00 
    - <RawMff | signal1.bin, 134 x 300683 (1202.7 s), ~307.6 MB, data loaded> 
    - time(s): 1202.732 
    - first_samp-last_samp: 0 - 300682
    - Duration = 0hr : 20min : 02sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 11  
 - 2397 / 250 = 9.588
 - 105013 / 250 = 420.052
 - 207642 / 250 = 830.568

________________________________________________________________________________________________
____________________________________________HC15______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20161026 1322 ----> 2016-10-26 18:47:34.780000+00:00 
    - <RawMff | signal1.bin, 133 x 222409 (889.6 s), ~225.9 MB, data loaded> 
    - time(s): 889.636 
    - first_samp-last_samp: 0 - 222408
    - Duration = 0hr : 14min : 49sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2685 / 250 = 10.74
 - 124834 / 250 = 499.336

________________________________________________________________________________________________
____________________________________________HC16______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20170315 1317 ----> 2017-03-15 18:37:41.816000+00:00 
    - <RawMff | signal1.bin, 133 x 219367 (877.5 s), ~222.8 MB, data loaded> 
    - time(s): 877.468 
    - first_samp-last_samp: 0 - 219366
    - Duration = 0hr : 14min : 37sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2333 / 250 = 9.332
 - 119100 / 250 = 476.4

________________________________________________________________________________________________
____________________________________________HC17______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20170329 1426 ----> 2017-03-29 19:43:25.788000+00:00 
    - <RawMff | signal1.bin, 133 x 247105 (988.4 s), ~250.9 MB, data loaded> 
    - time(s): 988.42 
    - first_samp-last_samp: 0 - 247104
    - Duration = 0hr : 16min : 28sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2769 / 250 = 11.076
 - 152691 / 250 = 610.764

________________________________________________________________________________________________
____________________________________________HC18______________________________________________


___________ Taken ___________
(1) Data timing... 
    - Session 20170330 1425 ----> 2017-03-30 19:40:47.532000+00:00 
    - <RawMff | signal1.bin, 133 x 227193 (908.8 s), ~230.7 MB, data loaded> 
    - time(s): 908.772 
    - first_samp-last_samp: 0 - 227192
    - Duration = 0hr : 15min : 08sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2618 / 250 = 10.472
 - 136853 / 250 = 547.412

________________________________________________________________________________________________
____________________________________________MSICU05______________________________________________


___________ IT02 ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230420 1042 ----> 2023-04-20 15:42:45.953000+00:00 
    - <RawMff | signal1.bin, 133 x 81532 (326.1 s), ~82.9 MB, data loaded> 
    - time(s): 326.128 
    - first_samp-last_samp: 0 - 81531
    - Duration = 0hr : 05min : 26sec
(2) Trigger Channels
    - Trigger ch. 32  
    - Trigger ch. 42  
 - 1804 / 250 = 7.216
 - 79899 / 250 = 319.596

___________ RS_ST_Taken ___________
(1) Data timing... 
    - Session 20220408 1321 ----> 2022-04-08 18:31:36.824000+00:00 
    - <RawMff | signal1.bin, 135 x 424286 (1697.1 s), ~437.2 MB, data loaded> 
    - time(s): 1697.144 
    - first_samp-last_samp: 0 - 424285
    - Duration = 0hr : 28min : 17sec
(2) Trigger Channels
    - Trigger ch. 9   
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1553 / 250 = 6.212
 - 156523 / 250 = 626.092
 - 244982 / 250 = 979.928
 - 340549 / 250 = 1362.196

___________ rest02 ___________

    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230420 1006 ----> 2023-04-20 15:21:53.396000+00:00 
    - <RawMff | signal1.bin, 133 x 84470 (337.9 s), ~85.9 MB, data loaded> 
    - time(s): 337.88 
    - first_samp-last_samp: 0 - 84469
    - Duration = 0hr : 05min : 37sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1593 / 250 = 6.372
 - 84107 / 250 = 336.428

___________ ST02 ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230420 1028 ----> 2023-04-20 15:30:54.493000+00:00 
    - <RawMff | signal1.bin, 133 x 87544 (350.2 s), ~89.0 MB, data loaded> 
    - time(s): 350.176 
    - first_samp-last_samp: 0 - 87543
    - Duration = 0hr : 05min : 50sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1812 / 250 = 7.248
 - 77476 / 250 = 309.904

________________________________________________________________________________________________
____________________________________________MSICU07______________________________________________


___________ RS_ST_Taken_Tennis ___________
(1) Data timing... 
    - Session 20220607 1636 ----> 2022-06-07 21:44:25.524000+00:00 
    - <RawMff | signal1.bin, 135 x 496770 (1987.1 s), ~511.8 MB, data loaded> 
    - time(s): 1987.08 
    - first_samp-last_samp: 0 - 496769
    - Duration = 0hr : 33min : 07sec
(2) Trigger Channels
    - Trigger ch. 9   
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1558 / 250 = 6.232
 - 150059 / 250 = 600.236
 - 250099 / 250 = 1000.396
 - 427043 / 250 = 1708.172

________________________________________________________________________________________________
____________________________________________MSICU09______________________________________________


___________ taken ___________
(1) Data timing... 
    - Session 20220726 1357 ----> 2022-07-26 18:59:23.460000+00:00 
    - <RawMff | signal1.bin, 133 x 175570 (702.3 s), ~178.3 MB, data loaded> 
    - time(s): 702.28 
    - first_samp-last_samp: 0 - 175569
    - Duration = 0hr : 11min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 959 / 250 = 3.836
 - 94125 / 250 = 376.5

___________ taken02 ___________
(1) Data timing... 
    - Session 20220728 1242 ----> 2022-07-28 17:45:27.760000+00:00 
    - <RawMff | signal1.bin, 133 x 172419 (689.7 s), ~175.1 MB, data loaded> 
    - time(s): 689.676 
    - first_samp-last_samp: 0 - 172418
    - Duration = 0hr : 11min : 29sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 944 / 250 = 3.776
 - 90050 / 250 = 360.2

___________ rest ___________
(1) Data timing... 
    - Session 20220726 1339 ----> 2022-07-26 18:42:54.728000+00:00 
    - <RawMff | signal1.bin, 132 x 91334 (365.3 s), ~92.2 MB, data loaded> 
    - time(s): 365.336 
    - first_samp-last_samp: 0 - 91333
    - Duration = 0hr : 06min : 05sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1472 / 250 = 5.888

___________ rest02 ___________
(1) Data timing... 
    - Session 20220728 1226 ----> 2022-07-28 17:29:57.772000+00:00 
    - <RawMff | signal1.bin, 132 x 90085 (360.3 s), ~90.9 MB, data loaded> 
    - time(s): 360.34 
    - first_samp-last_samp: 0 - 90084
    - Duration = 0hr : 06min : 00sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1495 / 250 = 5.98

________________________________________________________________________________________________
____________________________________________MSICU10______________________________________________


___________ rest ___________
(1) Data timing... 
    - Session 20220727 0858 ----> 2022-07-27 14:09:49.536000+00:00 
    - <RawMff | signal1.bin, 132 x 90123 (360.5 s), ~90.9 MB, data loaded> 
    - time(s): 360.492 
    - first_samp-last_samp: 0 - 90122
    - Duration = 0hr : 06min : 00sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1523 / 250 = 6.092

___________ taken ___________
(1) Data timing... 
    - Session 20220727 0926 ----> 2022-07-27 14:32:38.148000+00:00 
    - <RawMff | signal1.bin, 133 x 206319 (825.3 s), ~209.5 MB, data loaded> 
    - time(s): 825.276 
    - first_samp-last_samp: 0 - 206318
    - Duration = 0hr : 13min : 45sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 12  
 - 1254 / 250 = 5.016
 - 8803 / 250 = 35.212
 - 124083 / 250 = 496.332

________________________________________________________________________________________________
____________________________________________MSICU14______________________________________________


___________ rest ___________
(1) Data timing... 
    - Session 20010207 0240 ----> 2001-02-07 07:42:09.316000+00:00 
    - <RawMff | signal1.bin, 132 x 103796 (415.2 s), ~104.7 MB, data loaded> 
    - time(s): 415.184 
    - first_samp-last_samp: 0 - 103795
    - Duration = 0hr : 06min : 55sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1015 / 250 = 4.06

___________ taken ___________
(1) Data timing... 
    - Session 20010207 0250 ----> 2001-02-07 07:57:06.704000+00:00 
    - <RawMff | signal1.bin, 133 x 170032 (680.1 s), ~172.7 MB, data loaded> 
    - time(s): 680.128 
    - first_samp-last_samp: 0 - 170031
    - Duration = 0hr : 11min : 20sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 960 / 250 = 3.84
 - 87985 / 250 = 351.94

________________________________________________________________________________________________
____________________________________________MSICU15______________________________________________


___________ rest ___________
(1) Data timing... 
    - Session 20010219 1732 ----> 2001-02-19 22:34:20.568000+00:00 
    - <RawMff | signal1.bin, 132 x 99347 (397.4 s), ~100.2 MB, data loaded> 
    - time(s): 397.388 
    - first_samp-last_samp: 0 - 99346
    - Duration = 0hr : 06min : 37sec
(2) Trigger Channels
    - Trigger ch. 9   
 - 1006 / 250 = 4.024

___________ taken ___________
(1) Data timing... 
    - Session 20010219 1747 ----> 2001-02-19 22:50:53.076000+00:00 
    - <RawMff | signal1.bin, 133 x 171207 (684.8 s), ~173.9 MB, data loaded> 
    - time(s): 684.828 
    - first_samp-last_samp: 0 - 171206
    - Duration = 0hr : 11min : 24sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 961 / 250 = 3.844
 - 88643 / 250 = 354.572

________________________________________________________________________________________________
____________________________________________MSICU16______________________________________________


___________ rest ___________
(1) Data timing... 
    - Session 20010227 0443 ----> 2001-02-27 09:54:08.540000+00:00 
    - <RawMff | signal1.bin, 132 x 103585 (414.3 s), ~104.5 MB, data loaded> 
    - time(s): 414.34 
    - first_samp-last_samp: 0 - 103584
    - Duration = 0hr : 06min : 54sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1426 / 250 = 5.704

___________ taken ___________
(1) Data timing... 
    - Session 20010227 0510 ----> 2001-02-27 10:12:08.872000+00:00 
    - <RawMff | signal1.bin, 133 x 171594 (686.4 s), ~174.3 MB, data loaded> 
    - time(s): 686.376 
    - first_samp-last_samp: 0 - 171593
    - Duration = 0hr : 11min : 26sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 942 / 250 = 3.768
 - 90063 / 250 = 360.252

________________________________________________________________________________________________
____________________________________________MSICU18______________________________________________


___________ RS_ST_Taken ___________
(1) Data timing... 
    - Session 20221212 1639 ----> 2022-12-12 21:47:13.448000+00:00 
    - <RawMff | signal1.bin, 137 x 250316 (1001.3 s), ~261.8 MB, data loaded> 
    - time(s): 1001.264 
    - first_samp-last_samp: 0 - 250315
    - Duration = 0hr : 16min : 41sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1379 / 250 = 5.516
 - 88936 / 250 = 355.744
 - 91475 / 250 = 365.9
 - 167095 / 250 = 668.38
 - 169634 / 250 = 678.536
 - 247684 / 250 = 990.736

________________________________________________________________________________________________
____________________________________________MSICU19______________________________________________


___________ RS_ST_Taken ___________

    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230131 1717 ----> 2023-01-31 22:24:48.016000+00:00 
    - <RawMff | signal1.bin, 137 x 244504 (978.0 s), ~255.7 MB, data loaded> 
    - time(s): 978.016 
    - first_samp-last_samp: 0 - 244503
    - Duration = 0hr : 16min : 18sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1554 / 250 = 6.216
 - 84074 / 250 = 336.296
 - 86617 / 250 = 346.468
 - 162282 / 250 = 649.128
 - 164830 / 250 = 659.32
 - 242879 / 250 = 971.516

________________________________________________________________________________________________
____________________________________________MSICU21______________________________________________


___________ RS_ST_Taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230205 1031 ----> 2023-02-05 15:33:39.517000+00:00 
    - <RawMff | signal1.bin, 137 x 244492 (978.0 s), ~255.7 MB, data loaded> 
    - time(s): 977.968 
    - first_samp-last_samp: 0 - 244491
    - Duration = 0hr : 16min : 17sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1529 / 250 = 6.116
 - 84046 / 250 = 336.184
 - 86592 / 250 = 346.368
 - 162256 / 250 = 649.024
 - 164807 / 250 = 659.228
 - 242855 / 250 = 971.42

________________________________________________________________________________________________
____________________________________________MSICU22______________________________________________


___________ RS_ST_Taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230208 1712 ----> 2023-02-08 22:13:19.416000+00:00 
    - <RawMff | signal1.bin, 137 x 244492 (978.0 s), ~255.7 MB, data loaded> 
    - time(s): 977.968 
    - first_samp-last_samp: 0 - 244491
    - Duration = 0hr : 16min : 17sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1525 / 250 = 6.1
 - 84042 / 250 = 336.168
 - 86588 / 250 = 346.352
 - 162252 / 250 = 649.008
 - 164807 / 250 = 659.228
 - 242855 / 250 = 971.42

________________________________________________________________________________________________
____________________________________________MSICU26______________________________________________


___________ RS_ST_Taken ___________
(1) Data timing... 
    - Session 20230406 1651 ----> 2023-04-06 22:00:37.164000+00:00 
    - <RawMff | signal1.bin, 137 x 244470 (977.9 s), ~255.7 MB, data loaded> 
    - time(s): 977.88 
    - first_samp-last_samp: 0 - 244469
    - Duration = 0hr : 16min : 17sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1521 / 250 = 6.084
 - 84038 / 250 = 336.152
 - 86583 / 250 = 346.332
 - 162247 / 250 = 648.988
 - 164796 / 250 = 659.184
 - 242844 / 250 = 971.376

________________________________________________________________________________________________
____________________________________________MSICU28______________________________________________


___________ RS_ST_Taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230515 1622 ----> 2023-05-15 21:22:37.007000+00:00 
    - <RawMff | signal1.bin, 137 x 244495 (978.0 s), ~255.7 MB, data loaded> 
    - time(s): 977.98 
    - first_samp-last_samp: 0 - 244494
    - Duration = 0hr : 16min : 17sec
(2) Trigger Channels
    - Trigger ch. 59  
    - Trigger ch. 69  
    - Trigger ch. 62  
    - Trigger ch. 72  
    - Trigger ch. 60  
    - Trigger ch. 70  
 - 1508 / 250 = 6.032
 - 84027 / 250 = 336.108
 - 86603 / 250 = 346.412
 - 162270 / 250 = 649.08
 - 164812 / 250 = 659.248
 - 242862 / 250 = 971.448

________________________________________________________________________________________________
____________________________________________MSICU29______________________________________________


___________ RS_ST_Taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230609 1229 ----> 2023-06-09 17:56:09.966000+00:00 
    - <RawMff | signal1.bin, 137 x 244508 (978.0 s), ~255.7 MB, data loaded> 
    - time(s): 978.032 
    - first_samp-last_samp: 0 - 244507
    - Duration = 0hr : 16min : 18sec
(2) Trigger Channels
    - Trigger ch. 59  
    - Trigger ch. 69  
    - Trigger ch. 62  
    - Trigger ch. 72  
    - Trigger ch. 60  
    - Trigger ch. 70  
 - 1552 / 250 = 6.208
 - 84074 / 250 = 336.296
 - 86615 / 250 = 346.46
 - 162281 / 250 = 649.124
 - 164826 / 250 = 659.304
 - 242875 / 250 = 971.5

________________________________________________________________________________________________
____________________________________________MSICU30______________________________________________


___________ RS_ST_Taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230614 1437 ----> 2023-06-14 19:52:12.138000+00:00 
    - <RawMff | signal1.bin, 137 x 244495 (978.0 s), ~255.7 MB, data loaded> 
    - time(s): 977.98 
    - first_samp-last_samp: 0 - 244494
    - Duration = 0hr : 16min : 17sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1520 / 250 = 6.08
 - 84037 / 250 = 336.148
 - 86604 / 250 = 346.416
 - 162268 / 250 = 649.072
 - 164815 / 250 = 659.26
 - 242864 / 250 = 971.456

________________________________________________________________________________________________
____________________________________________NETICU01______________________________________________


___________ sedon1taken ___________
(1) Data timing... 
    - Session 20200304 1111 ----> 2020-03-04 16:18:51.748000+00:00 
    - <RawMff | signal1.bin, 133 x 190730 (762.9 s), ~193.7 MB, data loaded> 
    - time(s): 762.92 
    - first_samp-last_samp: 0 - 190729
    - Duration = 0hr : 12min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2670 / 250 = 10.68
 - 104505 / 250 = 418.02

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20200304 1229 ----> 2020-03-04 17:34:36.256000+00:00 
    - <RawMff | signal1.bin, 133 x 175181 (700.7 s), ~177.9 MB, data loaded> 
    - time(s): 700.724 
    - first_samp-last_samp: 0 - 175180
    - Duration = 0hr : 11min : 40sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2043 / 250 = 8.172
 - 84319 / 250 = 337.276

___________ sedon1rest ___________
(1) Data timing... 
    - Session 20200304 1047 ----> 2020-03-04 15:54:29.144000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

___________ sedoffrest ___________
(1) Data timing... 
    - Session 20200304 1216 ----> 2020-03-04 17:18:12.316000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU02______________________________________________


___________ sedoffrest ___________
(1) Data timing... 
    - Session 20200306 1030 ----> 2020-03-06 15:33:54.764000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20200306 0935 ----> 2020-03-06 14:41:16.368000+00:00 
    - <RawMff | signal1.bin, 133 x 185144 (740.6 s), ~188.0 MB, data loaded> 
    - time(s): 740.576 
    - first_samp-last_samp: 0 - 185143
    - Duration = 0hr : 12min : 20sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2778 / 250 = 11.112
 - 103678 / 250 = 414.712

___________ sedon1rest ___________

(1) Data timing... 
    - Session 20200306 0914 ----> 2020-03-06 14:24:38.580000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20200306 1012 ----> 2020-03-06 15:18:10.308000+00:00 
    - <RawMff | signal1.bin, 133 x 173268 (693.1 s), ~176.0 MB, data loaded> 
    - time(s): 693.072 
    - first_samp-last_samp: 0 - 173267
    - Duration = 0hr : 11min : 33sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 12  
 - 2081 / 250 = 8.324
 - 90725 / 250 = 362.9

________________________________________________________________________________________________
____________________________________________NETICU03______________________________________________


___________ sedon1rest ___________
(1) Data timing... 
    - Session 20200306 1804 ----> 2020-03-06 23:33:02.168000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20200306 1914 ----> 2020-03-07 00:21:29.552000+00:00 
    - <RawMff | signal1.bin, 133 x 178931 (715.7 s), ~181.7 MB, data loaded> 
    - time(s): 715.724 
    - first_samp-last_samp: 0 - 178930
    - Duration = 0hr : 11min : 55sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2040 / 250 = 8.16
 - 88278 / 250 = 353.112

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20200306 1844 ----> 2020-03-06 23:46:16.668000+00:00 
    - <RawMff | signal1.bin, 133 x 188356 (753.4 s), ~191.3 MB, data loaded> 
    - time(s): 753.424 
    - first_samp-last_samp: 0 - 188355
    - Duration = 0hr : 12min : 33sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 2675 / 250 = 10.7
 - 97108 / 250 = 388.432

___________ sedoffrest ___________
(1) Data timing... 
    - Session 20200306 1933 ----> 2020-03-07 00:35:02.716000+00:00 
    - <RawMff | signal1.bin, 131 x 150000 (600.0 s), ~150.1 MB, data loaded> 
    - time(s): 600.0 
    - first_samp-last_samp: 0 - 149999
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU04______________________________________________


___________ sedon1taken ___________
(1) Data timing... 
    - Session 20371028 1019 ----> 2037-10-28 15:30:24.148000+00:00 
    - <RawMff | signal1.bin, 133 x 174993 (700.0 s), ~177.7 MB, data loaded> 
    - time(s): 699.972 
    - first_samp-last_samp: 0 - 174992
    - Duration = 0hr : 11min : 39sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1365 / 250 = 5.46
 - 90339 / 250 = 361.356

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20371028 1133 ----> 2037-10-28 16:36:34.650000+00:00 
    - <RawMff | signal1.bin, 131 x 150470 (601.9 s), ~150.6 MB, data loaded> 
    - time(s): 601.88 
    - first_samp-last_samp: 0 - 150469
    - Duration = 0hr : 10min : 01sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20371028 1116 ----> 2037-10-28 16:21:01.752000+00:00 
    - <RawMff | signal1.bin, 133 x 180256 (721.0 s), ~183.1 MB, data loaded> 
    - time(s): 721.024 
    - first_samp-last_samp: 0 - 180255
    - Duration = 0hr : 12min : 01sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 929 / 250 = 3.716
 - 95972 / 250 = 383.888

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20371028 0957 ----> 2037-10-28 15:08:08.708000+00:00 
    - <RawMff | signal1.bin, 131 x 151220 (604.9 s), ~151.3 MB, data loaded> 
    - time(s): 604.88 
    - first_samp-last_samp: 0 - 151219
    - Duration = 0hr : 10min : 04sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU05______________________________________________


___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20210217 1130 ----> 2021-02-17 16:32:26.357000+00:00 
    - <RawMff | signal1.bin, 131 x 152644 (610.6 s), ~152.7 MB, data loaded> 
    - time(s): 610.576 
    - first_samp-last_samp: 0 - 152643
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20210217 1037 ----> 2021-02-17 15:43:32.004000+00:00 
    - <RawMff | signal1.bin, 133 x 306437 (1225.7 s), ~311.1 MB, data loaded> 
    - time(s): 1225.748 
    - first_samp-last_samp: 0 - 306436
    - Duration = 0hr : 20min : 25sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1557 / 250 = 6.228
 - 228342 / 250 = 913.368

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20210217 1014 ----> 2021-02-17 15:26:23.850000+00:00 
    - <RawMff | signal1.bin, 131 x 152670 (610.7 s), ~152.8 MB, data loaded> 
    - time(s): 610.68 
    - first_samp-last_samp: 0 - 152669
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20210217 1111 ----> 2021-02-17 16:18:30.356000+00:00 
    - <RawMff | signal1.bin, 133 x 161369 (645.5 s), ~163.9 MB, data loaded> 
    - time(s): 645.476 
    - first_samp-last_samp: 0 - 161368
    - Duration = 0hr : 10min : 45sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 923 / 250 = 3.692
 - 81736 / 250 = 326.944

________________________________________________________________________________________________
____________________________________________NETICU06______________________________________________


___________ sedon1taken ___________
(1) Data timing... 
    - Session 20210311 2137 ----> 2021-03-12 02:40:27.676000+00:00 
    - <RawMff | signal1.bin, 133 x 180781 (723.1 s), ~183.6 MB, data loaded> 
    - time(s): 723.124 
    - first_samp-last_samp: 0 - 180780
    - Duration = 0hr : 12min : 03sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1570 / 250 = 6.28
 - 98634 / 250 = 394.536

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20210311 2213 ----> 2021-03-12 03:15:50.670000+00:00 
    - <RawMff | signal1.bin, 131 x 150345 (601.4 s), ~150.4 MB, data loaded> 
    - time(s): 601.38 
    - first_samp-last_samp: 0 - 150344
    - Duration = 0hr : 10min : 01sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20210311 2226 ----> 2021-03-12 03:28:34.472000+00:00 
    - <RawMff | signal1.bin, 133 x 172581 (690.3 s), ~175.3 MB, data loaded> 
    - time(s): 690.324 
    - first_samp-last_samp: 0 - 172580
    - Duration = 0hr : 11min : 30sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 927 / 250 = 3.708
 - 88923 / 250 = 355.692

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20210311 2110 ----> 2021-03-12 02:26:06.565000+00:00 
    - <RawMff | signal1.bin, 131 x 153108 (612.4 s), ~153.2 MB, data loaded> 
    - time(s): 612.432 
    - first_samp-last_samp: 0 - 153107
    - Duration = 0hr : 10min : 12sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU08______________________________________________


___________ sedon2rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20211001 1545 ----> 2021-10-01 20:51:53.849000+00:00 
    - <RawMff | signal1.bin, 131 x 150407 (601.6 s), ~150.5 MB, data loaded> 
    - time(s): 601.628 
    - first_samp-last_samp: 0 - 150406
    - Duration = 0hr : 10min : 01sec
(2) Trigger Channels

___________ sedofftaken ___________

(1) Data timing... 
    - Session 20211001 1518 ----> 2021-10-01 20:28:05.460000+00:00 
    - <RawMff | signal1.bin, 134 x 172007 (688.0 s), ~176.0 MB, data loaded> 
    - time(s): 688.028 
    - first_samp-last_samp: 0 - 172006
    - Duration = 0hr : 11min : 28sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 12  
    - Trigger ch. 15  
 - 1808 / 250 = 7.232
 - 83277 / 250 = 333.108
 - 168965 / 250 = 675.86

___________ sedon2taken ___________
(1) Data timing... 
    - Session 20211001 1602 ----> 2021-10-01 21:03:42.724000+00:00 
    - <RawMff | signal1.bin, 134 x 169005 (676.0 s), ~173.0 MB, data loaded> 
    - time(s): 676.02 
    - first_samp-last_samp: 0 - 169004
    - Duration = 0hr : 11min : 16sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 15  
 - 918 / 250 = 3.672
 - 81453 / 250 = 325.812
 - 164497 / 250 = 657.988

________________________________________________________________________________________________
____________________________________________NETICU11______________________________________________


___________ sedon1taken ___________
(1) Data timing... 
    - Session 20211116 1059 ----> 2021-11-16 16:03:48.024000+00:00 
    - <RawMff | signal1.bin, 134 x 176456 (705.8 s), ~180.6 MB, data loaded> 
    - time(s): 705.824 
    - first_samp-last_samp: 0 - 176455
    - Duration = 0hr : 11min : 45sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 13  
 - 1673 / 250 = 6.692
 - 84674 / 250 = 338.696
 - 170077 / 250 = 680.308

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20211116 1118 ----> 2021-11-16 16:29:13.595000+00:00 
    - <RawMff | signal1.bin, 131 x 153444 (613.8 s), ~153.5 MB, data loaded> 
    - time(s): 613.776 
    - first_samp-last_samp: 0 - 153443
    - Duration = 0hr : 10min : 13sec
(2) Trigger Channels

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20211116 1042 ----> 2021-11-16 15:48:08.411000+00:00 
    - <RawMff | signal1.bin, 131 x 150645 (602.6 s), ~150.7 MB, data loaded> 
    - time(s): 602.58 
    - first_samp-last_samp: 0 - 150644
    - Duration = 0hr : 10min : 02sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20211116 1140 ----> 2021-11-16 16:42:56.520000+00:00 
    - <RawMff | signal1.bin, 134 x 170118 (680.5 s), ~174.1 MB, data loaded> 
    - time(s): 680.472 
    - first_samp-last_samp: 0 - 170117
    - Duration = 0hr : 11min : 20sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 13  
 - 1005 / 250 = 4.02
 - 81220 / 250 = 324.88
 - 164337 / 250 = 657.348

________________________________________________________________________________________________
____________________________________________NETICU13______________________________________________


___________ sedon1taken ___________
(1) Data timing... 
    - Session 20220329 0715 ----> 2022-03-29 12:21:29.460000+00:00 
    - <RawMff | signal1.bin, 134 x 178032 (712.1 s), ~182.2 MB, data loaded> 
    - time(s): 712.128 
    - first_samp-last_samp: 0 - 178031
    - Duration = 0hr : 11min : 52sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 13  
 - 1477 / 250 = 5.908
 - 83968 / 250 = 335.872
 - 173565 / 250 = 694.26

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20220329 0640 ----> 2022-03-29 12:03:51.583000+00:00 
    - <RawMff | signal1.bin, 131 x 150445 (601.8 s), ~150.5 MB, data loaded> 
    - time(s): 601.78 
    - first_samp-last_samp: 0 - 150444
    - Duration = 0hr : 10min : 01sec
(2) Trigger Channels

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20220329 0800 ----> 2022-03-29 13:03:34.360000+00:00 
    - <RawMff | signal1.bin, 134 x 181831 (727.3 s), ~186.1 MB, data loaded> 
    - time(s): 727.324 
    - first_samp-last_samp: 0 - 181830
    - Duration = 0hr : 12min : 07sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 13  
 - 928 / 250 = 3.712
 - 86130 / 250 = 344.52
 - 179011 / 250 = 716.044

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20220329 0734 ----> 2022-03-29 12:48:11.424000+00:00 
    - <RawMff | signal1.bin, 131 x 157920 (631.7 s), ~158.0 MB, data loaded> 
    - time(s): 631.68 
    - first_samp-last_samp: 0 - 157919
    - Duration = 0hr : 10min : 31sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU15______________________________________________


___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20220526 1730 ----> 2022-05-26 22:32:00.072000+00:00 
    - <RawMff | signal1.bin, 131 x 142245 (569.0 s), ~142.3 MB, data loaded> 
    - time(s): 568.98 
    - first_samp-last_samp: 0 - 142244
    - Duration = 0hr : 09min : 28sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20220526 1701 ----> 2022-05-26 22:08:45.408000+00:00 
    - <RawMff | signal1.bin, 134 x 185544 (742.2 s), ~189.9 MB, data loaded> 
    - time(s): 742.176 
    - first_samp-last_samp: 0 - 185543
    - Duration = 0hr : 12min : 22sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 13  
 - 1567 / 250 = 6.268
 - 92911 / 250 = 371.644
 - 182087 / 250 = 728.348

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20220526 1808 ----> 2022-05-26 23:10:48.996000+00:00 
    - <RawMff | signal1.bin, 134 x 185106 (740.4 s), ~189.4 MB, data loaded> 
    - time(s): 740.424 
    - first_samp-last_samp: 0 - 185105
    - Duration = 0hr : 12min : 20sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
    - Trigger ch. 13  
 - 933 / 250 = 3.732
 - 82592 / 250 = 330.368
 - 181464 / 250 = 725.856

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20220526 1744 ----> 2022-05-26 22:51:41.952000+00:00 
    - <RawMff | signal1.bin, 131 x 225368 (901.5 s), ~225.4 MB, data loaded> 
    - time(s): 901.472 
    - first_samp-last_samp: 0 - 225367
    - Duration = 0hr : 15min : 01sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU17______________________________________________


___________ sedofftaken ___________
(1) Data timing... 
    - Session 20220624 1231 ----> 2022-06-24 17:34:22.164000+00:00 
    - <RawMff | signal1.bin, 133 x 165132 (660.5 s), ~167.7 MB, data loaded> 
    - time(s): 660.528 
    - first_samp-last_samp: 0 - 165131
    - Duration = 0hr : 11min : 00sec
(2) Trigger Channels
    - Trigger ch. 10  
    - Trigger ch. 12  
 - 1086 / 250 = 4.344
 - 83420 / 250 = 333.68

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20220624 1116 ----> 2022-06-24 16:25:51.347000+00:00 
    - <RawMff | signal1.bin, 131 x 184319 (737.3 s), ~184.4 MB, data loaded> 
    - time(s): 737.276 
    - first_samp-last_samp: 0 - 184318
    - Duration = 0hr : 12min : 17sec
(2) Trigger Channels

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20220624 1218 ----> 2022-06-24 17:19:25.612000+00:00 
    - <RawMff | signal1.bin, 131 x 159870 (639.5 s), ~160.0 MB, data loaded> 
    - time(s): 639.48 
    - first_samp-last_samp: 0 - 159869
    - Duration = 0hr : 10min : 39sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20220624 1139 ----> 2022-06-24 16:43:49.468000+00:00 
    - <RawMff | signal1.bin, 134 x 196406 (785.6 s), ~201.0 MB, data loaded> 
    - time(s): 785.624 
    - first_samp-last_samp: 0 - 196405
    - Duration = 0hr : 13min : 05sec
(2) Trigger Channels
    - Trigger ch. 21  
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1639 / 250 = 6.556
 - 7749 / 250 = 30.996
 - 103503 / 250 = 414.012

________________________________________________________________________________________________
____________________________________________NETICU18______________________________________________


___________ sedon1taken ___________

(1) Data timing... 
    - Session 20010123 1936 ----> 2001-01-24 00:39:50.516000+00:00 
    - <RawMff | signal1.bin, 133 x 175182 (700.7 s), ~177.9 MB, data loaded> 
    - time(s): 700.728 
    - first_samp-last_samp: 0 - 175181
    - Duration = 0hr : 11min : 40sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1542 / 250 = 6.168
 - 93548 / 250 = 374.192

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010123 2010 ----> 2001-01-24 01:12:57.519000+00:00 
    - <RawMff | signal1.bin, 131 x 315139 (1260.6 s), ~315.1 MB, data loaded> 
    - time(s): 1260.556 
    - first_samp-last_samp: 0 - 315138
    - Duration = 0hr : 21min : 00sec
(2) Trigger Channels

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010123 1953 ----> 2001-01-24 00:58:32.258000+00:00 
    - <RawMff | signal1.bin, 131 x 150120 (600.5 s), ~150.2 MB, data loaded> 
    - time(s): 600.48 
    - first_samp-last_samp: 0 - 150119
    - Duration = 0hr : 10min : 00sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU19______________________________________________


___________ sedon1rest ___________
(1) Data timing... 
    - Session 20010124 1538 ----> 2001-01-24 20:39:53.472000+00:00 
    - <RawMff | signal1.bin, 131 x 225105 (900.4 s), ~225.2 MB, data loaded> 
    - time(s): 900.42 
    - first_samp-last_samp: 0 - 225104
    - Duration = 0hr : 15min : 00sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20010124 1517 ----> 2001-01-24 20:26:25.796000+00:00 
    - <RawMff | signal1.bin, 133 x 169182 (676.7 s), ~171.9 MB, data loaded> 
    - time(s): 676.728 
    - first_samp-last_samp: 0 - 169181
    - Duration = 0hr : 11min : 16sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1017 / 250 = 4.068
 - 87329 / 250 = 349.316

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20010124 1639 ----> 2001-01-24 21:41:01.168000+00:00 
    - <RawMff | signal1.bin, 133 x 169107 (676.4 s), ~171.8 MB, data loaded> 
    - time(s): 676.428 
    - first_samp-last_samp: 0 - 169106
    - Duration = 0hr : 11min : 16sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 951 / 250 = 3.804
 - 87243 / 250 = 348.972

___________ sedoffrest ___________
(1) Data timing... 
    - Session 20010124 1558 ----> 2001-01-24 20:59:08.420000+00:00 
    - <RawMff | signal1.bin, 131 x 154045 (616.2 s), ~154.1 MB, data loaded> 
    - time(s): 616.18 
    - first_samp-last_samp: 0 - 154044
    - Duration = 0hr : 10min : 16sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU20______________________________________________


___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010212 2348 ----> 2001-02-13 04:49:23.114000+00:00 
    - <RawMff | signal1.bin, 131 x 150782 (603.1 s), ~150.9 MB, data loaded> 
    - time(s): 603.128 
    - first_samp-last_samp: 0 - 150781
    - Duration = 0hr : 10min : 03sec
(2) Trigger Channels

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010212 2319 ----> 2001-02-13 04:20:24.872000+00:00 
    - <RawMff | signal1.bin, 131 x 360112 (1440.4 s), ~360.1 MB, data loaded> 
    - time(s): 1440.448 
    - first_samp-last_samp: 0 - 360111
    - Duration = 0hr : 24min : 00sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20010212 2258 ----> 2001-02-13 04:06:39.545000+00:00 
    - <RawMff | signal1.bin, 133 x 170870 (683.5 s), ~173.6 MB, data loaded> 
    - time(s): 683.48 
    - first_samp-last_samp: 0 - 170869
    - Duration = 0hr : 11min : 23sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 1377 / 250 = 5.508
 - 86870 / 250 = 347.48

________________________________________________________________________________________________
____________________________________________NETICU22______________________________________________


___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010322 0427 ----> 2001-03-22 09:37:18.005000+00:00 
    - <RawMff | signal1.bin, 133 x 249941 (999.8 s), ~253.8 MB, data loaded> 
    - time(s): 999.764 
    - first_samp-last_samp: 0 - 249940
    - Duration = 0hr : 16min : 39sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 936 / 250 = 3.744
 - 73845 / 250 = 295.38
 - 166858 / 250 = 667.432

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010322 0459 ----> 2001-03-22 10:08:23.525000+00:00 
    - <RawMff | signal1.bin, 132 x 157795 (631.2 s), ~159.1 MB, data loaded> 
    - time(s): 631.18 
    - first_samp-last_samp: 0 - 157794
    - Duration = 0hr : 10min : 31sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 934 / 250 = 3.736

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20010322 0525 ----> 2001-03-22 10:27:26.826000+00:00 
    - <RawMff | signal1.bin, 132 x 451796 (1807.2 s), ~455.2 MB, data loaded> 
    - time(s): 1807.184 
    - first_samp-last_samp: 0 - 451795
    - Duration = 0hr : 30min : 07sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 944 / 250 = 3.776

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20010322 0558 ----> 2001-03-22 11:08:26.744000+00:00 
    - <RawMff | signal1.bin, 133 x 185931 (743.7 s), ~188.8 MB, data loaded> 
    - time(s): 743.724 
    - first_samp-last_samp: 0 - 185930
    - Duration = 0hr : 12min : 23sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 10  
 - 998 / 250 = 3.992
 - 100846 / 250 = 403.384

________________________________________________________________________________________________
____________________________________________NETICU24______________________________________________


___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230112 1715 ----> 2023-01-12 22:29:34.426000+00:00 
    - <RawMff | signal1.bin, 135 x 160733 (642.9 s), ~165.7 MB, data loaded> 
    - time(s): 642.932 
    - first_samp-last_samp: 0 - 160732
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1814 / 250 = 7.256
 - 77485 / 250 = 309.94
 - 80036 / 250 = 320.144
 - 158090 / 250 = 632.36

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230112 1741 ----> 2023-01-12 22:47:26.506000+00:00 
    - <RawMff | signal1.bin, 132 x 333713 (1334.8 s), ~336.3 MB, data loaded> 
    - time(s): 1334.852 
    - first_samp-last_samp: 0 - 333712
    - Duration = 0hr : 22min : 14sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1266 / 250 = 5.064

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230112 1811 ----> 2023-01-12 23:18:24.552000+00:00 
    - <RawMff | signal1.bin, 133 x 152807 (611.2 s), ~155.2 MB, data loaded> 
    - time(s): 611.228 
    - first_samp-last_samp: 0 - 152806
    - Duration = 0hr : 10min : 11sec
(2) Trigger Channels
    - Trigger ch. 18  
    - Trigger ch. 28  
 - 1256 / 250 = 5.024
 - 151447 / 250 = 605.788

___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230112 1829 ----> 2023-01-12 23:31:04.419000+00:00 
    - <RawMff | signal1.bin, 135 x 160607 (642.4 s), ~165.6 MB, data loaded> 
    - time(s): 642.428 
    - first_samp-last_samp: 0 - 160606
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 48  
    - Trigger ch. 58  
    - Trigger ch. 40  
    - Trigger ch. 50  
 - 1740 / 250 = 6.96
 - 77367 / 250 = 309.468
 - 79917 / 250 = 319.668
 - 157970 / 250 = 631.88

________________________________________________________________________________________________
____________________________________________NETICU25______________________________________________


___________ sedon1taken ___________

(1) Data timing... 
    - Session 20230120 1549 ----> 2023-01-20 20:56:27.012000+00:00 
    - <RawMff | signal1.bin, 135 x 160869 (643.5 s), ~165.9 MB, data loaded> 
    - time(s): 643.476 
    - first_samp-last_samp: 0 - 160868
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1901 / 250 = 7.604
 - 77623 / 250 = 310.492
 - 80175 / 250 = 320.7
 - 158232 / 250 = 632.928

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230120 1608 ----> 2023-01-20 21:14:07.617000+00:00 
    - <RawMff | signal1.bin, 131 x 152620 (610.5 s), ~152.7 MB, data loaded> 
    - time(s): 610.48 
    - first_samp-last_samp: 0 - 152619
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230120 1625 ----> 2023-01-20 21:29:46.560000+00:00 
    - <RawMff | signal1.bin, 131 x 303394 (1213.6 s), ~303.4 MB, data loaded> 
    - time(s): 1213.576 
    - first_samp-last_samp: 0 - 303393
    - Duration = 0hr : 20min : 13sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU27______________________________________________


___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230419 0650 ----> 2023-04-19 12:05:03.697000+00:00 
    - <RawMff | signal1.bin, 135 x 160666 (642.7 s), ~165.7 MB, data loaded> 
    - time(s): 642.664 
    - first_samp-last_samp: 0 - 160665
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1700 / 250 = 6.8
 - 77366 / 250 = 309.464
 - 79989 / 250 = 319.956
 - 158040 / 250 = 632.16

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230419 0716 ----> 2023-04-19 12:17:41.207000+00:00 
    - <RawMff | signal1.bin, 134 x 334039 (1336.2 s), ~341.7 MB, data loaded> 
    - time(s): 1336.156 
    - first_samp-last_samp: 0 - 334038
    - Duration = 0hr : 22min : 16sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 99  
    - Trigger ch. 19  
 - 1259 / 250 = 5.036
 - 182681 / 250 = 730.724
 - 332685 / 250 = 1330.74

___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230419 0808 ----> 2023-04-19 13:10:42.998000+00:00 
    - <RawMff | signal1.bin, 135 x 160614 (642.5 s), ~165.6 MB, data loaded> 
    - time(s): 642.456 
    - first_samp-last_samp: 0 - 160613
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 48  
    - Trigger ch. 58  
    - Trigger ch. 40  
    - Trigger ch. 50  
 - 1731 / 250 = 6.924
 - 77395 / 250 = 309.58
 - 79944 / 250 = 319.776
 - 157992 / 250 = 631.968

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230419 0740 ----> 2023-04-19 12:56:43.790000+00:00 
    - <RawMff | signal1.bin, 133 x 152889 (611.6 s), ~155.3 MB, data loaded> 
    - time(s): 611.556 
    - first_samp-last_samp: 0 - 152888
    - Duration = 0hr : 10min : 11sec
(2) Trigger Channels
    - Trigger ch. 18  
    - Trigger ch. 28  
 - 1521 / 250 = 6.084
 - 151531 / 250 = 606.124

________________________________________________________________________________________________
____________________________________________NETICU28______________________________________________


___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230426 1436 ----> 2023-04-26 19:37:58.907000+00:00 
    - <RawMff | signal1.bin, 135 x 160589 (642.4 s), ~165.6 MB, data loaded> 
    - time(s): 642.356 
    - first_samp-last_samp: 0 - 160588
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 48  
    - Trigger ch. 58  
    - Trigger ch. 40  
    - Trigger ch. 50  
 - 1704 / 250 = 6.816
 - 77369 / 250 = 309.476
 - 79913 / 250 = 319.652
 - 157960 / 250 = 631.84

___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230426 1336 ----> 2023-04-26 18:37:50.704000+00:00 
    - <RawMff | signal1.bin, 135 x 160576 (642.3 s), ~165.6 MB, data loaded> 
    - time(s): 642.304 
    - first_samp-last_samp: 0 - 160575
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1736 / 250 = 6.944
 - 77354 / 250 = 309.416
 - 79902 / 250 = 319.608
 - 157950 / 250 = 631.8

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230426 1422 ----> 2023-04-26 19:25:24.451000+00:00 
    - <RawMff | signal1.bin, 133 x 152877 (611.5 s), ~155.3 MB, data loaded> 
    - time(s): 611.508 
    - first_samp-last_samp: 0 - 152876
    - Duration = 0hr : 10min : 11sec
(2) Trigger Channels
    - Trigger ch. 18  
    - Trigger ch. 28  
 - 1514 / 250 = 6.056
 - 151522 / 250 = 606.088

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230426 1348 ----> 2023-04-26 18:49:54.103000+00:00 
    - <RawMff | signal1.bin, 132 x 317565 (1270.3 s), ~320.0 MB, data loaded> 
    - time(s): 1270.26 
    - first_samp-last_samp: 0 - 317564
    - Duration = 0hr : 21min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1270 / 250 = 5.08

________________________________________________________________________________________________
____________________________________________NETICU29______________________________________________


___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230428 1736 ----> 2023-04-28 22:46:28.526000+00:00 
    - <RawMff | signal1.bin, 131 x 153926 (615.7 s), ~154.0 MB, data loaded> 
    - time(s): 615.704 
    - first_samp-last_samp: 0 - 153925
    - Duration = 0hr : 10min : 15sec
(2) Trigger Channels

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230428 1812 ----> 2023-04-28 23:14:38.744000+00:00 
    - <RawMff | signal1.bin, 131 x 157288 (629.1 s), ~157.4 MB, data loaded> 
    - time(s): 629.152 
    - first_samp-last_samp: 0 - 157287
    - Duration = 0hr : 10min : 29sec
(2) Trigger Channels

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20230428 1716 ----> 2023-04-28 22:25:06.172000+00:00 
    - <RawMff | signal1.bin, 135 x 159326 (637.3 s), ~164.3 MB, data loaded> 
    - time(s): 637.304 
    - first_samp-last_samp: 0 - 159325
    - Duration = 0hr : 10min : 37sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1733 / 250 = 6.932
 - 77353 / 250 = 309.412
 - 79901 / 250 = 319.604
 - 157949 / 250 = 631.796

___________ sedofftaken ___________
(1) Data timing... 
    - Session 20230428 1826 ----> 2023-04-28 23:27:53.852000+00:00 
    - <RawMff | signal1.bin, 135 x 159001 (636.0 s), ~163.9 MB, data loaded> 
    - time(s): 636.004 
    - first_samp-last_samp: 0 - 159000
    - Duration = 0hr : 10min : 36sec
(2) Trigger Channels
    - Trigger ch. 48  
    - Trigger ch. 58  
    - Trigger ch. 40  
    - Trigger ch. 50  
 - 1745 / 250 = 6.98
 - 77412 / 250 = 309.648
 - 79954 / 250 = 319.816
 - 158003 / 250 = 632.012

________________________________________________________________________________________________
____________________________________________NETICU30______________________________________________


___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230526 1042 ----> 2023-05-26 15:43:19.104000+00:00 
    - <RawMff | signal1.bin, 135 x 160577 (642.3 s), ~165.6 MB, data loaded> 
    - time(s): 642.308 
    - first_samp-last_samp: 0 - 160576
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 48  
    - Trigger ch. 58  
    - Trigger ch. 40  
    - Trigger ch. 50  
 - 1699 / 250 = 6.796
 - 77363 / 250 = 309.452
 - 79903 / 250 = 319.612
 - 157951 / 250 = 631.804

___________ sedon1rest ___________

    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230526 1007 ----> 2023-05-26 15:08:26.312000+00:00 
    - <RawMff | signal1.bin, 132 x 341163 (1364.6 s), ~343.8 MB, data loaded> 
    - time(s): 1364.652 
    - first_samp-last_samp: 0 - 341162
    - Duration = 0hr : 22min : 44sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1267 / 250 = 5.068

___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230526 0943 ----> 2023-05-26 14:55:57.934000+00:00 
    - <RawMff | signal1.bin, 135 x 160664 (642.7 s), ~165.7 MB, data loaded> 
    - time(s): 642.656 
    - first_samp-last_samp: 0 - 160663
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 12  
    - Trigger ch. 22  
    - Trigger ch. 10  
    - Trigger ch. 20  
 - 1731 / 250 = 6.924
 - 77352 / 250 = 309.408
 - 79991 / 250 = 319.964
 - 158039 / 250 = 632.156

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20230526 1031 ----> 2023-05-26 15:32:15.257000+00:00 
    - <RawMff | signal1.bin, 133 x 152876 (611.5 s), ~155.3 MB, data loaded> 
    - time(s): 611.504 
    - first_samp-last_samp: 0 - 152875
    - Duration = 0hr : 10min : 11sec
(2) Trigger Channels
    - Trigger ch. 18  
    - Trigger ch. 28  
 - 1516 / 250 = 6.064
 - 151520 / 250 = 606.08





## NEW DATA (13 Source-folders & 37 subfolders) ##



________________________________________________________________________________________________
____________________________________________CCTC07______________________________________________


___________ rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240325 1414 ----> 2024-03-25 19:25:43.762000+00:00 
    - <RawMff | signal1.bin, 133 x 77658 (310.6 s), ~79.0 MB, data loaded> 
    - time(s): 310.632 
    - first_samp-last_samp: 0 - 77657
    - Duration = 0hr : 05min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1263 / 250 = 5.052
 - 76285 / 250 = 305.14

___________ taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240325 1431 ----> 2024-03-25 19:31:35.663000+00:00 
    - <RawMff | signal1.bin, 135 x 160838 (643.3 s), ~165.8 MB, data loaded> 
    - time(s): 643.352 
    - first_samp-last_samp: 0 - 160837
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1944 / 250 = 7.776
 - 77611 / 250 = 310.444
 - 80149 / 250 = 320.596
 - 158197 / 250 = 632.788

________________________________________________________________________________________________
____________________________________________CCTC07NETICU______________________________________________


___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240328 1423 ----> 2024-03-28 19:24:00.059000+00:00 
    - <RawMff | signal1.bin, 132 x 318464 (1273.9 s), ~320.9 MB, data loaded> 
    - time(s): 1273.856 
    - first_samp-last_samp: 0 - 318463
    - Duration = 0hr : 21min : 13sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1261 / 250 = 5.044

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240328 1453 ----> 2024-03-28 19:54:13.667000+00:00 
    - <RawMff | signal1.bin, 133 x 152640 (610.6 s), ~155.1 MB, data loaded> 
    - time(s): 610.56 
    - first_samp-last_samp: 0 - 152639
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1265 / 250 = 5.06
 - 151279 / 250 = 605.116

___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240328 1504 ----> 2024-03-28 20:06:55.751000+00:00 
    - <RawMff | signal1.bin, 135 x 160714 (642.9 s), ~165.7 MB, data loaded> 
    - time(s): 642.856 
    - first_samp-last_samp: 0 - 160713
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1839 / 250 = 7.356
 - 77504 / 250 = 310.016
 - 80025 / 250 = 320.1
 - 158073 / 250 = 632.292

___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240328 1411 ----> 2024-03-28 19:12:29.866000+00:00 
    - <RawMff | signal1.bin, 135 x 160826 (643.3 s), ~165.8 MB, data loaded> 
    - time(s): 643.304 
    - first_samp-last_samp: 0 - 160825
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1921 / 250 = 7.684
 - 77587 / 250 = 310.348
 - 80126 / 250 = 320.504
 - 158175 / 250 = 632.7

________________________________________________________________________________________________
____________________________________________MSICU32______________________________________________

___________ rest ___________
    - This directory already exists. 
    - Continue with existing file.
    - Cant rename file to .mff because Session 20240220 2041.mff file already exisits. 
    - Continuing with existing file
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240220 2041 ----> 2024-02-21 01:44:07.407000+00:00 
    - <RawMff | signal1.bin, 131 x 90668 (362.7 s), ~90.8 MB, data loaded> 
    - time(s): 362.672 
    - first_samp-last_samp: 0 - 90667
    - Duration = 0hr : 06min : 02sec
(2) Trigger Channels

___________ taken ___________
(1) Data timing... 
    - Session 20240220 2022 ----> 2024-02-21 01:30:46.400000+00:00 
    - <RawMff | signal1.bin, 135 x 160839 (643.4 s), ~165.8 MB, data loaded> 
    - time(s): 643.356 
    - first_samp-last_samp: 0 - 160838
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1937 / 250 = 7.748
 - 77606 / 250 = 310.424
 - 80146 / 250 = 320.584
 - 158193 / 250 = 632.772
________________________________________________________________________________________________
____________________________________________MSICU38______________________________________________

___________ taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240311 1457 ----> 2024-03-11 19:58:58.964000+00:00 
    - <RawMff | signal1.bin, 135 x 160801 (643.2 s), ~165.8 MB, data loaded> 
    - time(s): 643.204 
    - first_samp-last_samp: 0 - 160800
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1898 / 250 = 7.592
 - 77563 / 250 = 310.252
 - 80101 / 250 = 320.404
 - 158151 / 250 = 632.604

___________ rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240311 1443 ----> 2024-03-11 19:52:20.669000+00:00 
    - <RawMff | signal1.bin, 134 x 77645 (310.6 s), ~79.6 MB, data loaded> 
    - time(s): 310.58 
    - first_samp-last_samp: 0 - 77644
    - Duration = 0hr : 05min : 10sec
(2) Trigger Channels
    - Trigger ch. cal*
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1262 / 250 = 5.048
 - 1287 / 250 = 5.148
 - 2687 / 250 = 10.748
 - 4050 / 250 = 16.2
 - 76279 / 250 = 305.116

________________________________________________________________________________________________
____________________________________________MSICU42______________________________________________


___________ rest ___________
(1) Data timing... 
    - Session 20240417 1431 ----> 2024-04-17 19:33:32.460000+00:00 
    - <RawMff | signal1.bin, 133 x 90207 (360.8 s), ~91.7 MB, data loaded> 
    - time(s): 360.828 
    - first_samp-last_samp: 0 - 90206
    - Duration = 0hr : 06min : 00sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1264 / 250 = 5.056
 - 88839 / 250 = 355.356

___________ taken ___________
(1) Data timing... 
    - Session 20240417 1442 ----> 2024-04-17 19:44:33.396000+00:00 
    - <RawMff | signal1.bin, 135 x 160889 (643.6 s), ~165.9 MB, data loaded> 
    - time(s): 643.556 
    - first_samp-last_samp: 0 - 160888
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1984 / 250 = 7.936
 - 77651 / 250 = 310.604
 - 80189 / 250 = 320.756
 - 158238 / 250 = 632.952

________________________________________________________________________________________________
____________________________________________MSICU43______________________________________________


___________ rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240417 1005 ----> 2024-04-17 15:13:20.578000+00:00 
    - <RawMff | signal1.bin, 133 x 90119 (360.5 s), ~91.6 MB, data loaded> 
    - time(s): 360.476 
    - first_samp-last_samp: 0 - 90118
    - Duration = 0hr : 06min : 00sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1267 / 250 = 5.068
 - 88786 / 250 = 355.144

___________ taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240417 1019 ----> 2024-04-17 15:21:21.902000+00:00 
    - <RawMff | signal1.bin, 135 x 160876 (643.5 s), ~165.9 MB, data loaded> 
    - time(s): 643.504 
    - first_samp-last_samp: 0 - 160875
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1935 / 250 = 7.74
 - 77647 / 250 = 310.588
 - 80185 / 250 = 320.74
 - 158234 / 250 = 632.936

________________________________________________________________________________________________
____________________________________________MSICU47______________________________________________


___________ rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240423 1659 ----> 2024-04-23 22:07:22.452000+00:00 
    - <RawMff | signal1.bin, 133 x 134789 (539.2 s), ~137.0 MB, data loaded> 
    - time(s): 539.156 
    - first_samp-last_samp: 0 - 134788
    - Duration = 0hr : 08min : 59sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1313 / 250 = 5.252
 - 88829 / 250 = 355.316

___________ taken ___________

    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240423 1716 ----> 2024-04-23 22:18:15.888000+00:00 
    - <RawMff | signal1.bin, 135 x 160963 (643.8 s), ~166.0 MB, data loaded> 
    - time(s): 643.852 
    - first_samp-last_samp: 0 - 160962
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 2062 / 250 = 8.248
 - 77731 / 250 = 310.924
 - 80268 / 250 = 321.072
 - 158318 / 250 = 633.272

________________________________________________________________________________________________
____________________________________________NETICU31______________________________________________


___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240207 1807 ----> 2024-02-07 23:09:41.046000+00:00 
    - <RawMff | signal1.bin, 135 x 160852 (643.4 s), ~165.9 MB, data loaded> 
    - time(s): 643.408 
    - first_samp-last_samp: 0 - 160851
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1990 / 250 = 7.96
 - 77615 / 250 = 310.46
 - 80153 / 250 = 320.612
 - 158203 / 250 = 632.812

___________ sedon2rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240207 1821 ----> 2024-02-07 23:22:18.055000+00:00 
    - <RawMff | signal1.bin, 131 x 302540 (1210.2 s), ~302.6 MB, data loaded> 
    - time(s): 1210.16 
    - first_samp-last_samp: 0 - 302539
    - Duration = 0hr : 20min : 10sec
(2) Trigger Channels

___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240207 1713 ----> 2024-02-07 22:21:05.907000+00:00 
    - <RawMff | signal1.bin, 135 x 160776 (643.1 s), ~165.8 MB, data loaded> 
    - time(s): 643.104 
    - first_samp-last_samp: 0 - 160775
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1902 / 250 = 7.608
 - 77523 / 250 = 310.092
 - 80061 / 250 = 320.244
 - 158126 / 250 = 632.504

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240207 1743 ----> 2024-02-07 22:44:10.311000+00:00 
    - <RawMff | signal1.bin, 133 x 7688 (30.7 s), ~8.0 MB, data loaded> 
    - time(s): 30.752 
    - first_samp-last_samp: 0 - 7687
    - Duration = 0hr : 00min : 30sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1300 / 250 = 5.2
 - 6314 / 250 = 25.256

________________________________________________________________________________________________
____________________________________________NETICU33______________________________________________


___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240224 1128 ----> 2024-02-24 16:29:46.064000+00:00 
    - <RawMff | signal1.bin, 133 x 7649 (30.6 s), ~7.9 MB, data loaded> 
    - time(s): 30.596 
    - first_samp-last_samp: 0 - 7648
    - Duration = 0hr : 00min : 30sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1258 / 250 = 5.032
 - 6273 / 250 = 25.092

___________ sedon1taken ___________
(1) Data timing... 
    - Session 20240224 1048 ----> 2024-02-24 15:53:45.296000+00:00 
    - <RawMff | signal1.bin, 135 x 160951 (643.8 s), ~166.0 MB, data loaded> 
    - time(s): 643.804 
    - first_samp-last_samp: 0 - 160950
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 2037 / 250 = 8.148
 - 77707 / 250 = 310.828
 - 80245 / 250 = 320.98
 - 158294 / 250 = 633.176

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240224 1106 ----> 2024-02-24 16:07:53.599000+00:00 
    - <RawMff | signal1.bin, 131 x 300304 (1201.2 s), ~300.3 MB, data loaded> 
    - time(s): 1201.216 
    - first_samp-last_samp: 0 - 300303
    - Duration = 0hr : 20min : 01sec
(2) Trigger Channels

________________________________________________________________________________________________
____________________________________________NETICU38______________________________________________


___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240308 1020 ----> 2024-03-08 15:21:36.061000+00:00 
    - <RawMff | signal1.bin, 132 x 767032 (3068.1 s), ~772.6 MB, data loaded> 
    - time(s): 3068.128 
    - first_samp-last_samp: 0 - 767031
    - Duration = 0hr : 51min : 08sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1261 / 250 = 5.044

___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240308 1008 ----> 2024-03-08 15:09:23.206000+00:00 
    - <RawMff | signal1.bin, 135 x 160976 (643.9 s), ~166.0 MB, data loaded> 
    - time(s): 643.904 
    - first_samp-last_samp: 0 - 160975
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 2044 / 250 = 8.176
 - 77736 / 250 = 310.944
 - 80275 / 250 = 321.1
 - 158333 / 250 = 633.332

________________________________________________________________________________________________
____________________________________________NETICU40______________________________________________


___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240403 0820 ----> 2024-04-03 13:21:02.107000+00:00 
    - <RawMff | signal1.bin, 133 x 152627 (610.5 s), ~155.1 MB, data loaded> 
    - time(s): 610.508 
    - first_samp-last_samp: 0 - 152626
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1257 / 250 = 5.028
 - 151271 / 250 = 605.084

___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240403 0733 ----> 2024-04-03 12:33:33.810000+00:00 
    - <RawMff | signal1.bin, 135 x 160926 (643.7 s), ~165.9 MB, data loaded> 
    - time(s): 643.704 
    - first_samp-last_samp: 0 - 160925
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 2030 / 250 = 8.12
 - 77700 / 250 = 310.8
 - 80238 / 250 = 320.952
 - 158287 / 250 = 633.148

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240403 0744 ----> 2024-04-03 12:45:02.412000+00:00 
    - <RawMff | signal1.bin, 132 x 339700 (1358.8 s), ~342.3 MB, data loaded> 
    - time(s): 1358.8 
    - first_samp-last_samp: 0 - 339699
    - Duration = 0hr : 22min : 38sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1264 / 250 = 5.056


___________ sedofftaken ___________
    - This directory already exists. 
    - Continue with existing file.
    - Cannot unzip files because Session 20240403 0832 already exists.
    - Continue with existing file.
    - Cant rename file to .mff because Session 20240403 0832.mff file already exisits. 
    - Continuing with existing file
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240403 0832 ----> 2024-04-03 13:32:54.655000+00:00 
    - <RawMff | signal1.bin, 135 x 160776 (643.1 s), ~165.8 MB, data loaded> 
    - time(s): 643.104 
    - first_samp-last_samp: 0 - 160775
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1882 / 250 = 7.528
 - 77546 / 250 = 310.184
 - 80084 / 250 = 320.336
 - 158132 / 250 = 632.528


________________________________________________________________________________________________
____________________________________________NETICU41______________________________________________


___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240402 1435 ----> 2024-04-02 19:51:26.036000+00:00 
    - <RawMff | signal1.bin, 135 x 160938 (643.7 s), ~165.9 MB, data loaded> 
    - time(s): 643.752 
    - first_samp-last_samp: 0 - 160937
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1997 / 250 = 7.988
 - 77664 / 250 = 310.656
 - 80219 / 250 = 320.876
 - 158289 / 250 = 633.156

___________ sedon2taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240402 1628 ----> 2024-04-02 21:28:38.307000+00:00 
    - <RawMff | signal1.bin, 135 x 160714 (642.9 s), ~165.7 MB, data loaded> 
    - time(s): 642.856 
    - first_samp-last_samp: 0 - 160713
    - Duration = 0hr : 10min : 42sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 1830 / 250 = 7.32
 - 77494 / 250 = 309.976
 - 80015 / 250 = 320.06
 - 158064 / 250 = 632.256

___________ sedon2rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240402 1601 ----> 2024-04-02 21:06:41.133000+00:00 
    - <RawMff | signal1.bin, 133 x 302640 (1210.6 s), ~307.3 MB, data loaded> 
    - time(s): 1210.56 
    - first_samp-last_samp: 0 - 302639
    - Duration = 0hr : 20min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1268 / 250 = 5.072
 - 301283 / 250 = 1205.132

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240402 1502 ----> 2024-04-02 20:03:00.507000+00:00 
    - <RawMff | signal1.bin, 132 x 347062 (1388.2 s), ~349.7 MB, data loaded> 
    - time(s): 1388.248 
    - first_samp-last_samp: 0 - 347061
    - Duration = 0hr : 23min : 08sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1259 / 250 = 5.036

___________ sedofftaken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240402 1549 ----> 2024-04-02 20:50:00.707000+00:00 
    - <RawMff | signal1.bin, 138 x 160764 (643.1 s), ~169.4 MB, data loaded> 
    - time(s): 643.056 
    - first_samp-last_samp: 0 - 160763
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. cal*
    - Trigger ch. cal=
    - Trigger ch. cal-
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 663 / 250 = 2.652
 - 727 / 250 = 2.908
 - 1013 / 250 = 4.052
 - 1871 / 250 = 7.484
 - 77535 / 250 = 310.14
 - 80074 / 250 = 320.296
 - 158122 / 250 = 632.488

___________ sedoffrest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240402 1538 ----> 2024-04-02 20:39:09.659000+00:00 
    - <RawMff | signal1.bin, 133 x 152640 (610.6 s), ~155.1 MB, data loaded> 
    - time(s): 610.56 
    - first_samp-last_samp: 0 - 152639
    - Duration = 0hr : 10min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
    - Trigger ch. 19  
 - 1266 / 250 = 5.064
 - 151280 / 250 = 605.12

________________________________________________________________________________________________
____________________________________________NETICU46______________________________________________


___________ sedon1taken ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240422 1532 ----> 2024-04-22 20:54:05.369000+00:00 
    - <RawMff | signal1.bin, 135 x 160939 (643.8 s), ~165.9 MB, data loaded> 
    - time(s): 643.756 
    - first_samp-last_samp: 0 - 160938
    - Duration = 0hr : 10min : 43sec
(2) Trigger Channels
    - Trigger ch. 22  
    - Trigger ch. 32  
    - Trigger ch. 20  
    - Trigger ch. 30  
 - 2080 / 250 = 8.32
 - 77701 / 250 = 310.804
 - 80239 / 250 = 320.956
 - 158289 / 250 = 633.156

___________ sedon1rest ___________
    > Data was downsampled to 250Hz from 1000.0
(1) Data timing... 
    - Session 20240422 1605 ----> 2024-04-22 21:05:58.257000+00:00 
    - <RawMff | signal1.bin, 132 x 302653 (1210.6 s), ~305.0 MB, data loaded> 
    - time(s): 1210.612 
    - first_samp-last_samp: 0 - 302652
    - Duration = 0hr : 20min : 10sec
(2) Trigger Channels
    - Trigger ch. 09  
 - 1258 / 250 = 5.032

